<h3>The Battle of Neighborhoods (Cont'd)<h3/>
Capstone Project - IBM Applied Data Science
    
Build a park in Kuala Lumpur Metropolitan City of Malaysia

The objective of this project is to find the most suitable place to build a park in Kuala Lumpur city. This is very useful especially to the government as they are aiming to improve the air pollution as well as reduce the carbon emission rate. Besides, it will helps reducing the social and mental problem of the citizens as they can have a nice gateway from the hectic and bustling city life without having to travel too far.

First, lets get start with downloading the necessary libraries.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

In [2]:
! pip install beautifulsoup4 
! pip install lxml
! pip install request
! pip install geocoder # to get coordinates

from bs4 import BeautifulSoup #library to scrap info from html
import requests #library to handle requests
import csv #library to read/save csv
import pandas as pd # library for data analysis 
import geocoder #library for latitude and longitude

     |████████████████████████████████| 102kB 1.8MB/s ta 0:00:01
     |████████████████████████████████| 5.8MB 14.3MB/s eta 0:00:01     |█████████████████▏              | 3.1MB 14.3MB/s eta 0:00:01     |██████████████████████▏         | 4.0MB 14.3MB/s eta 0:00:01     |████████████████████████████▏   | 5.1MB 14.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/30/84/5f/484cfba678967ef58c16fce6890925d5c7172622f20111fbfd
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/c1/e3/c1/d02c8c58538853e4c9b78cadb74f6d5c5c370b48a69a7271aa
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/c3/c3/24/b5c132b537ab380c02d69e6bd4dec1f5db56b5fe19030473d7
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/d6/a4/78/01b20a9dc224dcc009fab669f7f27b943b8889c5150bd68d8a
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/23/7c/6e/f5b4e09d6596c8b8802b347e48f149031e2363368048f1347a
Successfully built request get post query-string public
     |██████████████████

#### 1. Build Kuala Lumpur's Neighbourhood dataframe

In [3]:
#scrap the webpage of wikipedia to get the list of Kuala Lumpur's neighbourhood

# make the GET request 
url = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(url,'lxml')

neighbourhood = [] #to input the list 
for name in soup.find_all('div',class_="mw-category")[0].find_all('li'):
    neighbourhood.append(name.text)

In [4]:
# instantiate the dataframe
df = pd.DataFrame({str('Neighbourhood'):neighbourhood})
print('row:',df.shape[0],'column:',df.shape[1],) #shape of dataframe
df.head()

row: 70 column: 1


,Neighbourhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


#### 2. Find the latitude and longitude of the neighbourhoods

In [5]:
#function to get the latitude and longitude of the neighbourhoods
def get_latlng(neighbourhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},Kuala Lumpur, Malaysia'.format(neighbourhood))
        lat_lng_coords = g.latlng
        
    return lat_lng_coords

In [6]:
#call the latitude and longitude of each neighbourhood
coords = [get_latlng(neighbourhood) for neighbourhood in df['Neighbourhood'].tolist()]
coords

[[3.0576900000000364, 101.74388000000005],
 [3.1531525006886456, 101.70041313127312],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1347800000000348, 101.67262000000005],
 [3.111020000000053, 101.66283000000004],
 [3.09898000000004, 101.73499000000004],
 [3.1357600000000616, 101.70837000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.057800000000043, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.152017197420035, 101.70102760046613],
 [3.129290000000026, 101.69892000000004],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.147980000000075, 101.66798000000006],
 [3.1387586696676304, 101.6840455304707],
 [3.1387586696676304, 101.6840455304707],
 [3.156685175474611, 101.69807

In [7]:
#create the latitude and longitude dataframe 
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_coords.head()

,Latitude,Longitude
0,3.057690,101.743880
1,3.153153,101.700413
2,3.190350,101.625450
3,3.103910,101.712260
4,3.072750,101.714610


In [8]:
#combining the two table
df[['Latitude', 'Longitude']] = df_coords[['Latitude', 'Longitude']]
df.head()

,Neighbourhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.153153,101.700413
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


In [9]:
#Use geopy library to get the latitude and longitude values of Kuala Lumpur.
address = 'Kuala Lumpur, MY'

geolocator = Nominatim(user_agent="MY_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur are 3.1516636, 101.6943028.


In [10]:
# create map of Kuala Lumpur using latitude and longitude values
map_kualalumpur = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kualalumpur)  
    
map_kualalumpur

#### 3. Explore neighbourhoods using Foursquare API

In [11]:
CLIENT_ID = 'OOXX3LJZCAIVI1UTJEXF2LV4URNIB2CHF2IVTTTWUSTNRUT0' # your Foursquare ID
CLIENT_SECRET = 'MYER34C4WNTROKJISKCZFVDVNIGWOIGUS1NC5L4C23Z1WPKR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OOXX3LJZCAIVI1UTJEXF2LV4URNIB2CHF2IVTTTWUSTNRUT0
CLIENT_SECRET:MYER34C4WNTROKJISKCZFVDVNIGWOIGUS1NC5L4C23Z1WPKR


In [12]:
#extracts the category of the venue
def getNearbyVenues(names, latitudes, longitudes, radius=2000, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
kl_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Alam Damai
Ampang, Kuala Lumpur
Bandar Menjalara
Bandar Sri Permaisuri
Bandar Tasik Selatan
Bandar Tun Razak
Bangsar
Bangsar Park
Bangsar South
Batu 11 Cheras
Batu, Kuala Lumpur
Brickfields
Bukit Bintang
Bukit Jalil
Bukit Kiara
Bukit Nanas
Bukit Petaling
Bukit Tunku
Cheras, Kuala Lumpur
Chow Kit
Damansara Heights
Damansara Town Centre
Damansara, Kuala Lumpur
Dang Wangi
Desa Petaling
Federal Hill, Kuala Lumpur
Happy Garden
Jalan Cochrane, Kuala Lumpur
Jalan Duta
Jinjang
Kampung Baru, Kuala Lumpur
Kampung Datuk Keramat
Kepong
KL Eco City
Kuchai Lama
Lembah Pantai
Maluri
Medan Tuanku
Miharja
Mont Kiara
Pantai Dalam
Pudu, Kuala Lumpur
Putrajaya
Salak South
Segambut
Semarak
Sentul Raya
Setapak
Setiawangsa
Shamelin
Sri Hartamas
Sri Petaling
Sungai Besi
Taman Bukit Maluri
Taman Cheras Hartamas
Taman Connaught
Taman Desa
Taman Ibukota
Taman Len Seng
Taman Melati
Taman Midah
Taman OUG
Taman P. Ramlee
Taman Sri Sinar
Taman Taynton View
Taman Tun Dr Ismail
Taman U-Thant
Taman Wahyu
Titiwangsa
Wan

In [14]:
print('row:',kl_venues.shape[0],'column:',kl_venues.shape[1])
kl_venues.head()

row: 6983 column: 7


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
2,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
3,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
4,Alam Damai,3.05769,101.74388,Restoran Ikbal,3.061134,101.750220,Restaurant


In [15]:
kl_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


In [16]:
print('There are {} uniques categories.'.format(len(kl_venues['Venue Category'].unique())))

There are 305 uniques categories.


#### 4. Analyze each neighbourhoods

In this section, we will add categories from Foursquare to our existing dataframe.
Refer to our objective, we want to analyze and explore the best place to build a park in Kuala Lumpur.
Therefore, we will only needed the related data which is the availability of parks arround the 
neighbourhood areas.

In [17]:
# one hot encoding
kl_onehot = pd.get_dummies(kl_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighbourhood'] = kl_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

kl_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Café,Camera Store,Campground,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tourist Inf

In [18]:
print('row:',kl_onehot.shape[0],'column:',kl_onehot.shape[1])

row: 6983 column: 306


In [19]:
#group all the categories in each neighbourhoods using mean values
kl_grouped = kl_onehot.groupby('Neighbourhood').mean().reset_index()
kl_grouped

,Neighbourhood,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Café,Camera Store,Campground,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tourist Inf

In [20]:
kl_grouped.shape

(70, 306)

In [21]:
#filter only neighbourhood with/without a park
kl_park = kl_grouped[["Neighbourhood","Park"]]
kl_park

,Neighbourhood,Park
0,Alam Damai,0.010000
1,"Ampang, Kuala Lumpur",0.020000
2,Bandar Menjalara,0.040000
3,Bandar Sri Permaisuri,0.000000
4,Bandar Tasik Selatan,0.000000
5,Bandar Tun Razak,0.000000
6,Bangsar,0.000000
7,Bangsar Park,0.000000
8,Bangsar South,0.000000
9,Batu 11 Cheras,0.000000


#### 5. Clustering the neighbourhoods

In this section, we will analyze the dataframe using clustering method. This method, led us to 
decide the suitable and the best place to build a park in Kuala Lumpur. 

In [22]:
# set number of clusters
kclusters = 3

kl_park_clustering = kl_park.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_park_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [23]:
# add clustering labels
kl_park.insert(0, 'Cluster Labels', kmeans.labels_)

kl_merged = df

# merge kl_grouped with kl_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_park.set_index('Neighbourhood'), on='Neighbourhood')

kl_merged # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,Park
0,Alam Damai,3.057690,101.743880,2,0.010000
1,"Ampang, Kuala Lumpur",3.153153,101.700413,1,0.020000
2,Bandar Menjalara,3.190350,101.625450,1,0.040000
3,Bandar Sri Permaisuri,3.103910,101.712260,0,0.000000
4,Bandar Tasik Selatan,3.072750,101.714610,0,0.000000
5,Bandar Tun Razak,3.082800,101.722810,0,0.000000
6,Bangsar,3.129200,101.678440,0,0.000000
7,Bangsar Park,3.134780,101.672620,0,0.000000
8,Bangsar South,3.111020,101.662830,0,0.000000
9,Batu 11 Cheras,3.098980,101.734990,0,0.000000


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighbourhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighbourhood,Latitude,Longitude,Cluster Labels,Park
3,Bandar Sri Permaisuri,3.103910,101.712260,0,0.0
4,Bandar Tasik Selatan,3.072750,101.714610,0,0.0
5,Bandar Tun Razak,3.082800,101.722810,0,0.0
6,Bangsar,3.129200,101.678440,0,0.0
7,Bangsar Park,3.134780,101.672620,0,0.0
8,Bangsar South,3.111020,101.662830,0,0.0
9,Batu 11 Cheras,3.098980,101.734990,0,0.0
10,"Batu, Kuala Lumpur",3.135760,101.708370,0,0.0
11,Brickfields,3.129160,101.684060,0,0.0
14,Bukit Kiara,3.143480,101.644330,0,0.0


In [30]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighbourhood,Latitude,Longitude,Cluster Labels,Park
1,"Ampang, Kuala Lumpur",3.153153,101.700413,1,0.020000
2,Bandar Menjalara,3.190350,101.625450,1,0.040000
15,Bukit Nanas,3.152017,101.701028,1,0.020000
17,Bukit Tunku,3.173810,101.682760,1,0.020000
21,Damansara Town Centre,3.138759,101.684046,1,0.030000
22,"Damansara, Kuala Lumpur",3.138759,101.684046,1,0.030000
23,Dang Wangi,3.156685,101.698077,1,0.020000
25,"Federal Hill, Kuala Lumpur",3.136370,101.685640,1,0.020000
28,Jalan Duta,3.180163,101.677880,1,0.020000
32,Kepong,3.217500,101.637630,1,0.020000


In [31]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighbourhood,Latitude,Longitude,Cluster Labels,Park
0,Alam Damai,3.057690,101.743880,2,0.010000
12,Bukit Bintang,3.147770,101.708550,2,0.010000
13,Bukit Jalil,3.057800,101.689650,2,0.010000
16,Bukit Petaling,3.129290,101.698920,2,0.010000
18,"Cheras, Kuala Lumpur",3.061870,101.746750,2,0.010000
20,Damansara Heights,3.147980,101.667980,2,0.010000
27,"Jalan Cochrane, Kuala Lumpur",3.132977,101.724671,2,0.010000
30,"Kampung Baru, Kuala Lumpur",3.165460,101.710280,2,0.010000
40,Pantai Dalam,3.094760,101.667470,2,0.010000
49,Shamelin,3.124580,101.735970,2,0.010000


In [32]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]

,Neighbourhood,Latitude,Longitude,Cluster Labels,Park


In [34]:
kl_merged.loc[kl_merged['Cluster Labels'] == 4]

,Neighbourhood,Latitude,Longitude,Cluster Labels,Park


From the cluster result, we can define that Cluster 4 is the neighbourhoods that do not have parks at all. While the rest of the clusters have parks and distinguished depending on the number of it. 
As shown in the cluster map above, we can see that the number of parks in Kuala Lumpur can be improved especially North and East of Kuala Lumpur which is relatively fewer compare to the other sides.
As for a conclusion, the suitable place to build a park in Kuala Lumpur is North and East of the city. 

In [35]:
d = kl_merged.loc[kl_merged['Cluster Labels'] == 0]
d.shape

(33, 5)